<a href="https://colab.research.google.com/github/Patric-Ramz/bdt-2023-26720051/blob/main/Day%203/BDT_Day3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.2 LTS
Release:	22.04
Codename:	jammy


In [2]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [515 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,158 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [49.9 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,137 kB]
Hit:

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [4]:
!wget https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

--2023-09-26 18:42:55--  https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400395283 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.0-bin-hadoop3.tgz’

spark-3.5.0-bin-had 100%[===================>] 381.85M  20.0MB/s    in 5.3s    

2023-09-26 18:43:01 (71.6 MB/s) - ‘spark-3.5.0-bin-hadoop3.tgz’ saved [400395283/400395283]



In [5]:
VERSION = "3.5.0"

In [6]:
# decompress spark
!tar xf spark-$VERSION-bin-hadoop3.tgz

# install python package to help with system paths
!pip install -q findspark

In [7]:
# Let Colab know where the java and spark folders are
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-{VERSION}-bin-hadoop3"

In [20]:
# add pyspark to sys.path using findspark
import findspark
findspark.init()

In [21]:
from pyspark.sql import SparkSession
import pandas as pd
spark = SparkSession.builder.appName('panda-and-spark').getOrCreate()

In [23]:
!curl -o rows.csv https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  411M    0  411M    0     0  2876k      0 --:--:--  0:02:26 --:--:-- 2637k


In [24]:
import pandas as pd
nyc_df = pd.read_csv('rows.csv')

<ipython-input-24-a70f860f02a9>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  nyc_df = pd.read_csv('rows.csv')


In [25]:
print(len(nyc_df))

2028253


In [26]:
nyc_small = nyc_df.sample(frac=0.2, replace=False, random_state=1)

In [27]:
nyc_small = nyc_small[['CRASH DATE', 'CONTRIBUTING FACTOR VEHICLE 1',
                       'BOROUGH', 'VEHICLE TYPE CODE 1',
                       'NUMBER OF PERSONS INJURED']]

In [28]:
nyc_small.head(2)

,CRASH DATE,CONTRIBUTING FACTOR VEHICLE 1,BOROUGH,VEHICLE TYPE CODE 1,NUMBER OF PERSONS INJURED
1394476,02/06/2015,Lost Consciousness,MANHATTAN,PASSENGER VEHICLE,0.0
44918,06/16/2021,Failure to Yield Right-of-Way,QUEENS,Sedan,1.0


In [29]:
from pyspark.sql import SQLContext


# there are nan's in the frame with strings, and spark can't 'infer the schema', so we have to help it out
# by replacing them with empty strings and forcing the column to be a string

sdf_small = SQLContext(spark).createDataFrame(nyc_small.fillna('').astype('str'))


# Lets check the schema quickly

print(sdf_small.schema)

/content/spark-3.5.0-bin-hadoop3/python/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


StructType([StructField('CRASH DATE', StringType(), True), StructField('CONTRIBUTING FACTOR VEHICLE 1', StringType(), True), StructField('BOROUGH', StringType(), True), StructField('VEHICLE TYPE CODE 1', StringType(), True), StructField('NUMBER OF PERSONS INJURED', StringType(), True)])


In [31]:
# Question 1

from pyspark.sql import functions as F

sdf_small.groupBy('CRASH DATE') \
    .agg(F.count('CRASH DATE').alias('count')) \
    .orderBy(F.desc('count')) \
    .show(5)

+----------+-----+
|CRASH DATE|count|
+----------+-----+
|01/21/2014|  246|
|11/15/2018|  214|
|02/03/2014|  204|
|01/18/2015|  202|
|04/28/2017|  196|
+----------+-----+
only showing top 5 rows



In [32]:
# Question 2

sdf_small.groupBy('BOROUGH') \
    .agg(F.count('BOROUGH').alias('count')) \
    .orderBy(F.desc('count')) \
    .show(5)

+---------+------+
|  BOROUGH| count|
+---------+------+
|         |126550|
| BROOKLYN| 88661|
|   QUEENS| 74491|
|MANHATTAN| 62967|
|    BRONX| 41335|
+---------+------+
only showing top 5 rows



In [33]:
# Question 3

sdf_small.filter(sdf_small.BOROUGH == 'QUEENS') \
    .groupBy('CONTRIBUTING FACTOR VEHICLE 1') \
    .agg(F.count('CONTRIBUTING FACTOR VEHICLE 1').alias('count')) \
    .orderBy(F.desc('count')) \
    .show(5)

+-----------------------------+-----+
|CONTRIBUTING FACTOR VEHICLE 1|count|
+-----------------------------+-----+
|                  Unspecified|26266|
|         Driver Inattentio...|15802|
|         Failure to Yield ...| 6301|
|             Backing Unsafely| 3663|
|         Following Too Clo...| 2681|
+-----------------------------+-----+
only showing top 5 rows



In [34]:
sdf_small.groupBy('VEHICLE TYPE CODE 1', 'BOROUGH') \
    .agg(F.avg('NUMBER OF PERSONS INJURED').alias('avg_injuries')) \
    .orderBy(F.desc('avg_injuries')) \
    .show(3)

+-------------------+--------+------------+
|VEHICLE TYPE CODE 1| BOROUGH|avg_injuries|
+-------------------+--------+------------+
|          FIRETRUCK|BROOKLYN|         6.0|
|              BOX T|BROOKLYN|         5.0|
|            mta bus|        |         3.5|
+-------------------+--------+------------+
only showing top 3 rows

